In [1]:
import numpy as np
import matplotlib.pyplot as plt
import GCN

In [2]:
adjacencyList = np.load("filtered_data/hseAdjacencyList.npy", allow_pickle=True)
inputFeatureVectorList = np.load("filtered_data/inputFeatureVector.npy")
outputLabelList = np.load("filtered_data/outputFeatureVector.npy", allow_pickle=True)

# adjacencyList = [np.array(adjacencyList[i], dtype=np.double).T for i in range(len(adjacencyList))]

adjMatrix = np.load("filtered_data/hseAdjacencyMatrix.npy", allow_pickle=True)

inputFeatureVectorList = inputFeatureVectorList.astype(np.double)

adjMatrix = np.where(adjMatrix < .25, 1, 0)



In [3]:
# reload the gcn lib
import GCN
# load the data
# load the data
from torch_geometric.utils import dense_to_sparse

from torch_geometric.loader import DataLoader
from torch_geometric.data import Data
import torch
from sklearn.model_selection import train_test_split

def trainModel(outputLabel, device, epochs, verbose=True, save=False):

    # should likely be done all as a preprocessing step however it takes so little of the time it doesnt matter that much

    labels = np.array([outputLabelList[i][outputLabel] for i in range(len(outputLabelList))], dtype=np.double)

    # normalize the labels
    labels = (labels - np.min(labels)) / (np.max(labels) - np.min(labels))


    # Split data into train and validation sets (70% train, 30% val)
    X_train, X_val, adj_train, adj_val, y_train, y_val = train_test_split(
        inputFeatureVectorList, adjMatrix, labels, test_size=0.3, random_state=42
    )

    # Convert everything to a tensor
    inputFeatureTensor_train = torch.tensor(X_train, dtype=torch.float).to(device)
    inputFeatureTensor_val = torch.tensor(X_val, dtype=torch.float).to(device)
    adjMatrixTensor_train = torch.tensor(adj_train, dtype=torch.float).to(device)
    adjMatrixTensor_val = torch.tensor(adj_val, dtype=torch.float).to(device)
    labelsTensor_train = torch.tensor(y_train, dtype=torch.float).to(device)
    labelsTensor_val = torch.tensor(y_val, dtype=torch.float).to(device)

    # Prepare datasets and loaders
    data_set_train = GCN.InterpreterDataset(
        inputFeatureTensor_train,
        adjMatrixTensor_train,
        labelsTensor_train,
    )
    data_set_val = GCN.InterpreterDataset(
        inputFeatureTensor_val,
        adjMatrixTensor_val,
        labelsTensor_val,
    )


    data_loader_train = DataLoader(data_set_train, batch_size=100, shuffle=True)
    data_loader_val = DataLoader(data_set_val, batch_size=100, shuffle=False)

    #print length of the dataset
    print(f"Training on {len(data_loader_train.dataset)} samples")
    print(f"Validating on {len(data_loader_val.dataset)} samples")

    model = GCN.GraphConv(len(inputFeatureVectorList[0][0]), .001).to(device)
    model.trainModel(data_loader_train, data_loader_val, epochs, verbose=verbose)

    if save:
        torch.save(model.state_dict(), f"./models/model_{outputLabel}.pt")

In [4]:
import importlib
importlib.reload(GCN)


trainLabelList = ["defect_fermi_energy"]

# trainLabelList =[
#     "defect_total_energy","defect_energy_convergence", "defect_fermi_energy", "defect_vbm_energy", 
#     "defect_cbm_energy", "defect_cbm-vbm_energy", "defect_spin_state", "defect_max_hf", 
#     "defect_max_hf_asymmetry", "defect_dos_at_fermi_energy", "defect_dos_at_vbm_energy",
#     "defect_dos_at_cbm_energy","defect_bulk_gap", "defect_lvl_delta", "defect_lvl_vbm_delta",
#     "defect_lvl_cbm_delta", "defect_num_occ_lvl", "defect_num_unocc_lvl","defect_occ_min_localization",
#     "defect_unocc_min_localization", "defect_occ_avg_localization", "defect_unocc_avg_localization",
#     "defect_lvl_tdm"]


# Use CUDA if available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

for label in trainLabelList:
    print(f"Training model for {label}")
    trainModel(label, device, 30, save=True, verbose=False)


cuda:0
Training model for defect_fermi_energy
Training on 5406 samples
Validating on 2317 samples


Epoch 1/30:   0%|          | 0/55 [00:00<?, ?it/s]/home/petedowney/miniconda3/envs/MLQuanta/lib/python3.10/site-packages/torch/nn/modules/loss.py:538: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([10000, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Epoch 1/30:  98%|█████████▊| 54/55 [00:03<00:00, 16.49it/s, loss=0.0416]/home/petedowney/miniconda3/envs/MLQuanta/lib/python3.10/site-packages/torch/nn/modules/loss.py:538: UserWarning: Using a target size (torch.Size([6])) that is different to the input size (torch.Size([600, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Epoch 1/30: 100%|██████████| 55/55 [00:03<00:00, 15.52it/s, loss=0.042] 
/home/petedowney/miniconda3/envs/MLQuanta/lib/pyt